# Variational Quantum Eigensolver (VQE)

VQE [[1]](https://arxiv.org/abs/2012.09265) is an algorithm that can be used to approximate the ground state of a given Hamiltonian, represented by the operator $H$. If the exact ground state $|\psi_0\rangle$ is unknown, the VQE algorithm can be used to obtain an approximation of the ground state $|\psi\rangle\approx|\psi_0\rangle$, by applying the variational principle.

To apply the variational principle, the VQE algorithm starts with a guess state, called an ansatz, which is parameterized by a set of variables represented by $\theta\equiv(\theta_1,\theta_2,\cdots)$. The goal is to find the values of the variables $\theta_i$'s that minimize the energy expectation value of the Hamiltonian $H$ for the ansatz state $|\psi(\theta)\rangle$, i.e., $\langle\psi(\theta)|H|\psi(\theta)\rangle$.

To understand the VQE algorithm, we will use the **Ising model** Hamiltonian as a test case. The Hamiltonian for the Ising model can be expressed as follows:

\begin{equation}
H=\sum_{i=1}^{N-1} \sigma_i^z\sigma_{i+1}^z
\end{equation}

Here, $N$ represents the total number of qubits in the system, and $\sigma_i^z$ and $\sigma_{i+1}^z$ are the Pauli-Z matrices acting on qubits $i$ and $i+1$, respectively.

In [1]:
#import necessary libraries
import qiskit
from qiskit.circuit.library import EfficientSU2
from qiskit.opflow import I, X, Y, Z
import numpy as np
import random
from scipy.optimize import minimize
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import Statevector

Now we will construct this Hamiltonian using Qiskit for $N=4$ as follows,

In [2]:
# Making a list of all interaction terms

lst=[("ZZII",1),("IZZI",1),("IIZZ",1)]   # Considering Open boundary conditions
H = qiskit.opflow.primitive_ops.PauliSumOp.from_list(lst)
print(H)

1.0 * ZZII
+ 1.0 * IZZI
+ 1.0 * IIZZ


Once we have constructed the Hamiltonian, we can construct the ansatz circuit for $|\psi(\theta)\rangle$ as follows. We can choose to use the EfficientSU2 ansatz, which is a commonly used ansatz in quantum algorithms. However, it is also possible to use other ansatzes depending on the user's preference and the specific Hamiltonian being considered (more information click [here](https://arxiv.org/abs/2111.05176)).

In [3]:
ansatz = EfficientSU2(4, entanglement='linear', reps=1, skip_final_rotation_layer=True).decompose()
print(ansatz.num_parameters)
ansatz.draw()
#  ## This finds number of independent parameters in a ansatz
# from qiskit.circuit.library import PauliTwoDesign
# ansatz = PauliTwoDesign(4, reps=1, seed=2)
# print(ansatz.num_parameters)

8


┌──────────┐┌──────────┐               
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├──■────────────
     ├──────────┤├──────────┤┌─┴─┐          
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├┤ X ├──■───────
     ├──────────┤├──────────┤└───┘┌─┴─┐     
q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├─────┤ X ├──■──
     ├──────────┤├──────────┤     └───┘┌─┴─┐
q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├──────────┤ X ├
     └──────────┘└──────────┘          └───┘

In VQE, our goal is to find the state corresponding to the minimum eigen energy. To achieve this, we define the cost function as the expectation value of the Hamiltonian with respect to $|\psi(\theta)\rangle$, which is represented as $\langle\psi(\theta)|H|\psi(\theta)\rangle$. The function for calculating the expectation value of the Hamiltonian is `expectation_h`.

To approximate the ground state, we optimize the parameters using classical optimization techniques. For this purpose, we use the **Scipy** library to minimize our cost function using the `minimization` function provided below. 

In [4]:
def minimization(H, ansatz):
    '''
    Returns a optimal paramters for the state corresponding to minimum energy
    
        Parameters:
        H: The Hamiltonian operator for the system to be simulated.
        ansatz: A parameterized circuit used as Ansatz for the wave function.
        
        Returns:
        optimal_parameters: list of the optimal parameters for the ansatz circuit.
    
    '''
    
    def expectation_h(parameters):
        # Create the ansatz circuit with the given parameters
        qc = ansatz.bind_parameters(parameters)

        # Calculate the expectation value of the Hamiltonian H
        c = Statevector.from_instruction(qc).expectation_value(H)

        # Return the energy
        return c.real
    
    # Define the initial parameters for the ansatz circuit as a random numbers
    initial_parameters = []
    for i in range(ansatz.num_parameters):
        initial_parameters.append(random.random())
    
    result = minimize(expectation_h, initial_parameters, method='CG')
    optimal_parameters = result.x
    return optimal_parameters

To approximate the most excited state (highest energy state), we optimize the parameters using classical optimization techniques. For this purpose, we use the **Scipy** library to maximize our cost function using the `maximization` function provided below. 

In [5]:
def maximization(H, ansatz):
    '''
    Returns a optimal paramters for the state corresponding to maximum energy
    
        Parameters:
        H: The Hamiltonian operator for the system to be simulated.
        ansatz: A parameterized circuit used as Ansatz for the wave function.
        
        Returns:
        optimal_parameters: list of the optimal parameters for the ansatz circuit.
    
    '''
    
    def expectation_h(parameters):
        # Create the ansatz circuit with the given parameters
        qc = ansatz.bind_parameters(parameters)

        # Calculate the expectation value of the Hamiltonian H
        c = Statevector.from_instruction(qc).expectation_value(H)

        # Return the energy
        return -c.real
    
    # Define the initial parameters for the ansatz circuit as a random numbers
    initial_parameters = []
    for i in range(ansatz.num_parameters):
        initial_parameters.append(random.random())
    
    result = minimize(expectation_h, initial_parameters, method='CG')
    optimal_parameters = result.x
    return optimal_parameters

In [6]:
optimal_parameters=minimization(H, ansatz)

In [7]:
circuit = ansatz.bind_parameters(optimal_parameters)
circuit.draw()
expectation = Statevector.from_instruction(circuit).expectation_value(H)
print(expectation.real)

-2.999999999999975


# Application to optimization problem

Here we will try to minimize the quadratic equations below,
\begin{eqnarray}
\min_x x^2-2x+1,\\
\min_x x^2-4x+4.\\
\end{eqnarray}

To optimize these quadratic equations, we need to encode them into a Hamiltonian. To do this, we perform the transformation $x\rightarrow (\mathbb{I}-Z)/2$, where $\mathbb{I}$ is the identity operator and $Z$ is the Pauli Z gate. Since these equations have only one variable, we only need one qubit. 
\begin{eqnarray*}
H(x^2 - 2x + 1)=0.5Z+0.5\mathbb{I},\\
H(x^2 - 4x + 4)=1.5Z+2.5\mathbb{I}.
\end{eqnarray*}

## Solving $\min_x x^2-2x+1$

In [12]:
lst=[("Z",0.5),("I",0.5)]   ## Hamiltonian 1/2 Z+ 1/2 I
H1= qiskit.opflow.primitive_ops.PauliSumOp.from_list(lst)
print(H1)
ansatz1 = EfficientSU2(1, entanglement='linear', reps=1, skip_final_rotation_layer=True).decompose()

0.5 * Z
+ 0.5 * I


In [13]:
optimal_parameters1=maximization(H1, ansatz1)
circuit = ansatz1.bind_parameters(optimal_parameters1)
circuit.draw()
expectation = Statevector.from_instruction(circuit).expectation_value(H1)
print(f'The root of the equation x^2-2x+1 is {abs(expectation)**0.5}')

The root of the equation x^2-2x+1 is 0.9999999999999984


## Solving $\min_x x^2-4x+4$

In [10]:
lst=[("Z",1.5),("I",2.5)]   ## Hamiltonian 3/2 Z+ 5/2 I
H2 = qiskit.opflow.primitive_ops.PauliSumOp.from_list(lst)
print(H2)
ansatz2 = EfficientSU2(1, entanglement='linear', reps=1, skip_final_rotation_layer=True).decompose()

1.5 * Z
+ 2.5 * I


In [11]:
optimal_parameters2=maximization(H2, ansatz2)
circuit = ansatz2.bind_parameters(optimal_parameters2)
circuit.draw()
expectation = Statevector.from_instruction(circuit).expectation_value(H2)
print(f'The root of the equation x^2-4x+4 is {abs(expectation)**0.5}')

The root of the equation x^2-4x+4 is 1.9999999999999998
